# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [1]:
from dotenv import load_dotenv

from consumer import get_db_connection

## Setup

In [2]:
load_dotenv()


True

### 1. What exhibition is most frequently visited?

In [3]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT exhibition_id, COUNT(exhibition_id) FROM rating
                 GROUP BY exhibition_id
                 ORDER BY count DESC;''')
    rows = curr.fetchone()
    id = rows['exhibition_id']
    count = rows['count']

with conn.cursor() as curr:
    curr.execute('''SELECT exhibition_id, exhibition_name FROM exhibition
                 WHERE exhibition_id = %s;''', (id ,))
    data = curr.fetchone()
    name = data['exhibition_name']

print(f'The "{name}" exhibition was most frequently visited, with {count} ratings.')

The "Our Polluted World" exhibition was most frequently visited, with 481 ratings.


### 2. What hour of the day has the most ratings?

In [4]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id), EXTRACT(hour from time_given) FROM rating
                 GROUP BY EXTRACT(hour from time_given)
                 ORDER BY COUNT(rating_id) DESC
                 ;''')
    data = curr.fetchall()[0]
    print(f'The hour starting at {data["extract"]}:00 has the most ratings, {data["count"]} ratings.')

The hour starting at 10:00 has the most ratings, 232 ratings.


### 3. What is the average rating for each exhibition?

In [5]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT rating.exhibition_id, exhibition.exhibition_name, ROUND(AVG(rating.rating_value_id),2) FROM rating
                 JOIN exhibition
                 ON exhibition.exhibition_id = rating.exhibition_id
                 GROUP BY exhibition.exhibition_name,rating.exhibition_id;''')
    rows = curr.fetchall()
    for row in rows:
        print(f'The "{row["exhibition_name"]}" exhibition was rated {row["round"]} on average.')
    print('\nAny exhibitions not mentioned in this list, have not received any ratings.')




The "The Crenshaw Collection" exhibition was rated 2.43 on average.
The "Adaptation" exhibition was rated 2.93 on average.
The "Our Polluted World" exhibition was rated 2.22 on average.
The "Measureless to Man" exhibition was rated 2.92 on average.
The "Cetacean Sensations" exhibition was rated 3.83 on average.

Any exhibitions not mentioned in this list, have not received any ratings.


### 4. What proportion of all 4+ ratings are given to exhibition 4?

In [6]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id) FROM rating
                 WHERE rating_value_id >= 5;''')
    total_four_plus = curr.fetchone()['count']
    
with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id) FROM rating
                 WHERE rating_value_id >= 5
                 AND exhibition_id = 5;''')
    exhibition_four = curr.fetchone()['count']

print(f'{exhibition_four} ratings of 4+ were given to Exhibition 4 out of {total_four_plus} total 4+ ratings.')
print(f'This is {round(exhibition_four/total_four_plus*100, 1)}%')

37 ratings of 4+ were given to Exhibition 4 out of 287 total 4+ ratings.
This is 12.9%


### 5. Are positive ratings more frequent before or after 1pm?

In [7]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id) FROM rating
                 WHERE rating_value_id >= 4
                 AND EXTRACT(hour from time_given) <13
                 ;''')
    before_one = curr.fetchone()["count"]


with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id) FROM rating
                 WHERE rating_value_id >= 4
                 AND EXTRACT(hour from time_given) >= 13
                 ;''')
    after_one = curr.fetchone()['count']

print(f'There are {before_one} positive ratings given before 1pm, and {after_one} positive ratings given after 1pm.')
print(f'So positive ratings are more frequent {"before" if before_one>after_one else "after"} 1pm.')

There are 313 positive ratings given before 1pm, and 344 positive ratings given after 1pm.
So positive ratings are more frequent after 1pm.


### 6. How many ratings each hour are above the average rating for exhibition 4?

In [8]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT AVG(rating_value_id) from rating
                 WHERE exhibition_id = 5
                 ;''')
    avg = curr.fetchone()["avg"]
    print(f'The average rating for Exhibition 4 is {round(avg, 2)}.\n')

with conn.cursor() as curr:
    curr.execute('''SELECT COUNT(rating_id), EXTRACT(hour from time_given) FROM rating
                 WHERE exhibition_id = 5
                 AND rating_value_id > %s
                 GROUP BY EXTRACT(hour from time_given)
                 ;''', (avg,))
    data = curr.fetchall()
    for row in data:
        print(f'The hour starting at {row["extract"]}:00 had {row["count"]} rating(s) above {round(avg, 2)}.')


The average rating for Exhibition 4 is 2.22.

The hour starting at 14:00 had 17 rating(s) above 2.22.
The hour starting at 18:00 had 5 rating(s) above 2.22.
The hour starting at 9:00 had 17 rating(s) above 2.22.
The hour starting at 13:00 had 20 rating(s) above 2.22.
The hour starting at 16:00 had 19 rating(s) above 2.22.
The hour starting at 10:00 had 21 rating(s) above 2.22.
The hour starting at 8:00 had 15 rating(s) above 2.22.
The hour starting at 17:00 had 16 rating(s) above 2.22.
The hour starting at 11:00 had 12 rating(s) above 2.22.
The hour starting at 15:00 had 17 rating(s) above 2.22.
The hour starting at 12:00 had 14 rating(s) above 2.22.


### 7. Do Zoology exhibitions get better ratings than other types?

In [9]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT department.department_name, ROUND(AVG(rating.rating_value_id),2) FROM rating
                 JOIN exhibition
                 ON exhibition.exhibition_id = rating.exhibition_id
                 JOIN department 
                 ON department.department_id = exhibition.department_id
                 GROUP BY exhibition.department_id, department.department_name;''')
    rows = curr.fetchall()
    for row in rows:
        print(f'The {row["department_name"]} Department has an average rating of {row["round"]}.')
    print('\nSo the Zoology Department does have better ratings than the other departments.')

The Entomology Department has an average rating of 2.93.
The Geology Department has an average rating of 2.92.
The Ecology Department has an average rating of 2.22.
The Zoology Department has an average rating of 3.20.

So the Zoology Department does have better ratings than the other departments.


### 8. What exhibition has the most emergencies?

In [10]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute(
        '''SELECT exhibition.exhibition_name, COUNT(action.action_id) FROM action
        JOIN exhibition
        ON action.exhibition_id = exhibition.exhibition_id
        WHERE action_type_id = 2
        GROUP BY exhibition.exhibition_name
        ORDER BY COUNT(action.action_id) DESC
        ;''')
    data = curr.fetchone()
    print(f'The "{data["exhibition_name"]}" Exhibition had the most emergencies, {data["count"]} emergencies.')

The "The Crenshaw Collection" Exhibition had the most emergencies, 2 emergencies.


### 9. Which exhibitions receive fewer assistance requests than the average?

In [11]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute(
        '''SELECT COUNT(exhibition_id) from exhibition;''')
    exhibitions = curr.fetchone()["count"]
    print(
        f'There are {exhibitions} exhibitions.')
    
with conn.cursor() as curr:
    curr.execute(
        '''SELECT COUNT(action.action_id) FROM action
        WHERE action_type_id = 1
        ;''')
    assistance = curr.fetchone()["count"]
avg = assistance/exhibitions
print(f'There were {assistance} assistance requests, meaning the average per exhibition is {round(avg,1)}.')
print('So the Exhibitions with less than this are as follows:\n')

with conn.cursor() as curr:
    curr.execute(
        '''SELECT exhibition.exhibition_name, COUNT(action.action_id) FROM action
        JOIN exhibition
        ON action.exhibition_id = exhibition.exhibition_id
        WHERE action_type_id = 1
        GROUP BY exhibition.exhibition_name
        HAVING (COUNT(action.action_id))< %s
        ORDER BY COUNT(action.action_id) DESC
        ;''', (avg,))
    data = curr.fetchall()
    for row in data:
        print(
            f'The "{row["exhibition_name"]}" Exhibition had {row["count"]} assisstance requests.')

There are 6 exhibitions.
There were 71 assistance requests, meaning the average per exhibition is 11.8.
So the Exhibitions with less than this are as follows:

The "Our Polluted World" Exhibition had 11 assisstance requests.
The "Adaptation" Exhibition had 10 assisstance requests.
The "Measureless to Man" Exhibition had 9 assisstance requests.


### 10. Are there particular times when assistance requests/emergencies are more likely?

In [12]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute(
        '''SELECT COUNT(action.action_id), EXTRACT(hour from time_occurred) FROM action
        GROUP BY EXTRACT(hour from time_occurred)
        ORDER BY COUNT(action.action_id) DESC
        ;''')
    data = curr.fetchall()[:3]
    print('The top three times for assistance requests/emergencies are:')
    for row in data:
        print(f'The hour of {row["extract"]}:00, when there were {row["count"]}.')
    

The top three times for assistance requests/emergencies are:
The hour of 14:00, when there were 11.
The hour of 16:00, when there were 9.
The hour of 10:00, when there were 8.


### 11. Are emergencies more likely in exhibitions that are more exciting than others?

With the current dataset I would suggest we do not have enough data to draw a conclusion about this question.  
_For now accept the null hypothesis._

In [13]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT rating.exhibition_id, exhibition.exhibition_name, ROUND(AVG(rating.rating_value_id),2) FROM rating
                 JOIN exhibition
                 ON exhibition.exhibition_id = rating.exhibition_id
                 GROUP BY exhibition.exhibition_name,rating.exhibition_id
                 ORDER BY AVG(rating.rating_value_id) DESC;''')
    rows = curr.fetchall()
    for row in rows:
        print(
            f'The "{row["exhibition_name"]}" exhibition was rated {row["round"]} on average.')
        
print('\n')
with conn.cursor() as curr:
    curr.execute(
        '''SELECT exhibition.exhibition_name, COUNT(action.action_id) FROM action
        JOIN exhibition
        ON action.exhibition_id = exhibition.exhibition_id
        WHERE action_type_id = 2
        GROUP BY exhibition.exhibition_name
        ORDER BY COUNT(action.action_id) DESC
        ;''')
    data = curr.fetchall()
    for row in data:
        print(
        f'The "{row["exhibition_name"]}" Exhibition had {row["count"]} emergencies.')

The "Cetacean Sensations" exhibition was rated 3.83 on average.
The "Adaptation" exhibition was rated 2.93 on average.
The "Measureless to Man" exhibition was rated 2.92 on average.
The "The Crenshaw Collection" exhibition was rated 2.43 on average.
The "Our Polluted World" exhibition was rated 2.22 on average.


The "The Crenshaw Collection" Exhibition had 2 emergencies.


### 12. Which floors are above average for ratings?

In [14]:
conn = get_db_connection()
with conn.cursor() as curr:
    curr.execute('''SELECT ROUND(AVG(rating_value_id),2) FROM rating;''')
    avg = curr.fetchone()["round"]
print(f'The average rating across all exhibitions is {avg}.')
print('Floors with a higher rating than this are as follows:\n')

with conn.cursor() as curr:
    curr.execute('''SELECT floor.floor_name, ROUND(AVG(rating.rating_value_id),2) FROM rating
                 JOIN exhibition
                 ON exhibition.exhibition_id = rating.exhibition_id
                 JOIN floor
                 ON exhibition.floor_id = floor.floor_id
                 GROUP BY floor.floor_id
                 HAVING AVG(rating.rating_value_id) > %s
                 ORDER BY AVG(rating.rating_value_id) DESC;''', (avg,))
    rows = curr.fetchall()
    for row in rows:
        print(f'{row["floor_name"]}, with an average rating of {row["round"]}.')

The average rating across all exhibitions is 2.87.
Floors with a higher rating than this are as follows:

Floor 1, with an average rating of 3.47.
Vault, with an average rating of 2.93.
